In [5]:
from drn_interactions.io import load_neurons, load_spikes
from drn_interactions.config import Config
from drn_interactions.stats import cv_isi_burst, is_burster, median_burst_interval

import seaborn as sns
import matplotlib.pyplot as plt

from pathlib import Path

In [6]:
outdir = Config.derived_data_dir / "neurons"
outdir.mkdir(exist_ok=True)

In [7]:
neurons = load_neurons()
df_spikes = load_spikes("pre")

In [8]:
neurons

,neuron_id,waveform_width,peak_asymmetry,mean_firing_rate,cv2_isi,recording_session_id,cluster_id,channel,is_single_unit,session_name,group_name,experiment_name
0,1,16.0,0.065574,0.601693,0.467315,3,0,6,1,ESHOCK_03_LOC1,shock,ESHOCK
1,4,9.0,0.037344,2.105512,0.824797,3,3,17,1,ESHOCK_03_LOC1,shock,ESHOCK
2,5,17.0,0.070539,1.778190,0.394575,3,4,20,1,ESHOCK_03_LOC1,shock,ESHOCK
3,6,21.0,0.087137,0.832163,0.428821,3,5,26,1,ESHOCK_03_LOC1,shock,ESHOCK
4,8,70.0,0.343137,1.690391,0.305202,3,7,28,1,ESHOCK_03_LOC1,shock,ESHOCK
...,...,...,...,...,...,...,...,...,...,...,...,...
561,2627,8.0,0.033333,0.916395,1.032022,57,52,18,1,acute_11,acute_saline,CITWAY
562,2628,19.0,0.078189,0.429663,1.250412,57,53,29,1,acute_11,acute_saline,CITWAY
563,2629,17.0,0.068826,2.536273,0.840159,57,54,30,1,acute_11,acute_saline,CITWAY
564,2630,67.0,0.233449,10.256485,0.573768,57,55,18,1,acute_11,acute_saline,CITWAY


In [4]:
cv2_burst = df_spikes.groupby("neuron_id")["spiketimes"].apply(cv_isi_burst, thresh=0.05)
is_burst = df_spikes.groupby("neuron_id")["spiketimes"].apply(is_burster, burst_thresh=0.05, frac_bursts=0.1)
med_burst = df_spikes.groupby("neuron_id")["spiketimes"].apply(median_burst_interval, thresh=0.05)

df = (
    neurons
    .merge(cv2_burst.to_frame(name="cv2_burst").reset_index())
    .merge(is_burst.to_frame(name="is_burster").reset_index())
    .merge(med_burst.to_frame(name="median_burst").reset_index())
)
df.head(2)


c:\Users\roryl\.conda\envs\drn-interactions\lib\site-packages\numpy\core\fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


,neuron_id,waveform_width,peak_asymmetry,mean_firing_rate,cv2_isi,recording_session_id,cluster_id,channel,is_single_unit,session_name,group_name,experiment_name,cv2_burst,is_burster,median_burst
0,1,16.0,0.065574,0.601693,0.467315,3,0,6,1,ESHOCK_03_LOC1,shock,ESHOCK,0.449766,True,0.041733
1,4,9.0,0.037344,2.105512,0.824797,3,3,17,1,ESHOCK_03_LOC1,shock,ESHOCK,0.638198,True,0.022283


In [5]:
df.to_csv(outdir / "burst_features.csv", index=False)